# Attention Architecture

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import music21

In [3]:
# from fastai.text import *
from enum import Enum
import torch

In [4]:
import numpy as np
import torch.nn as nn

In [5]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [6]:
import sys
sys.path.insert(0, '../../')
from src.fastai_data import *
from src.encode_data import *
from src.serve import *

In [7]:
from src.music_transformer import *

In [8]:
from src.unilm import *

In [9]:
config = v15s_config(vocab)

In [10]:
config['bs'] = 2
config['bptt'] = 1024
config['n_layers'] = 4
config['n_heads'] = 4
config['dec_layers'] = 2

In [11]:
config

{'ctx_len': 150,
 'n_layers': 4,
 'n_heads': 4,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 512,
 'mask': True,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 2,
 'bptt': 1024,
 'vocab_size': 274,
 'dec_layers': 2}

In [12]:
base_path = Path('../../data/midi/v15/')

In [13]:
cache_name = 'tmp/sample'

## BERT Dataloading

In [14]:
ns_dl_tfms = [mask_tfm, next_sentence_tfm]
ns_config = config.copy()
ns_config['bs'] *= 2
ns_data = load_music_data(base_path/'piano_duet', cache_name=cache_name, vocab=vocab, 
                          y_offset=0, dl_tfms=ns_dl_tfms, **ns_config)

## S2S Dataloading

In [15]:
s2s_dl_tfms = [mask_s2s_tfm]
s2s_data = MusicDataBunch.load(base_path/'s2s_encode', cache_name=cache_name, 
                           preloader_cls=S2SPreloader, dl_tfms=[mask_s2s_tfm], y_offset=1,
                           shuffle_dl=True, **config)


### Load NextWord dataset (Language Model)

In [16]:
nw_dl_tfms = [nw_tfm]
nw_data = load_music_data(base_path/'piano_duet', cache_name=cache_name, 
                          vocab=vocab, dl_tfms=nw_dl_tfms, y_offset=1, **config)

## Load

In [17]:
loss_func = BertLoss()
learn = bert_model_learner(ns_data, config.copy(), 
                           loss_func=loss_func)

Sep_idx: 8


In [18]:
learn.metrics = [mask_acc, ns_acc, s2s_acc, nw_acc]

In [19]:
learn.callbacks = [BertTrainer(learn, ns_data, s2s_data, nw_data)]
# learn.callbacks = [BertTrainer(learn, s2s_data, s2s_data, s2s_data)]

In [20]:
# learn.loss_func = BertLoss(CrossEntropyFlat(ignore_index=vocab.pad_idx), 
#                            CrossEntropyFlat(), CrossEntropyFlat(ignore_index=vocab.pad_idx))

In [21]:
learn = learn.to_fp16()

In [22]:
# learn.validate()

Switching to next sentence data


[8.449357, tensor(0.0975), tensor(0.0489), tensor(0), tensor(0)]

In [23]:
# learn.lr_find()
# learn.recorder.plot()

In [23]:
# xb,yb = s2s_data.one_batch(); xb[0].shape, yb[0].shape

(torch.Size([2, 1024]), torch.Size([2, 1024]))

In [22]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,mask_acc,ns_acc,s2s_acc,nw_acc,time
0,5.289381,4.925184,0.380158,0.676007,0.000000,0.000000,00:42
1,4.909459,4.633219,0.384515,0.685021,0.000000,0.000000,00:42
2,6.136724,6.181915,0.511495,0.000000,0.000000,0.000000,00:22
3,6.824258,6.735467,0.143384,0.000000,0.000000,0.111001,01:22
4,3.689878,3.564862,0.473932,0.601187,0.000000,0.000000,00:42


Switching to next sentence data
Switching to translate data
Switching to next word data
Switching to next sentence data
Switching to translate data
Switching to next word data


In [ ]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,mask_acc,ns_acc,s2s_acc,nw_acc,time
0,4.013850,3.958144,0.365504,0.634691,0.000000,0.000000,00:41
1,5.777857,6.053072,0.534073,0.000000,0.030359,0.000000,00:21
2,6.567859,6.494505,0.292803,0.000000,0.000000,0.251255,01:20
3,3.649584,3.456672,0.493514,0.656701,0.000000,0.000000,00:41


Switching to translate data
Switching to next word data
Switching to next sentence data
Switching to translate data
